# Places for Half Earth Future
#### This is a new feature that will show precalculated biodiversity and contextual data for selected geometries around the globe (Up to 20 places per country)
#### Following this first approach there might be more narrative involved and a diffeent approach to how to show the data. Right now, the layer will be accessed from the 'Explore places' tab.

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
import json
import pandas as pd
from arcgis.features import FeatureLayerCollection
import requests as re
from copy import deepcopy

In [ ]:
#Columba livia, SliceNumber = 2233
#gadm1= pd.read_csv('/Users/tamarahuete/Documents/HALF_EARTH/gadm1_precalc_no_geometry.csv')
#json.loads(gadm1[gadm1['NAME_1']=='Nyeri']['birds'].values[0])
#{'SliceNumber': 2330.0, 'percentage_presence': 85.8}

In [24]:
env_path = "../../.env"
with open(env_path) as f:
   env = {}
   for line in f:
       env_key, _val = line.split("=")
       env_value = _val.split("\n")[0]
       env[env_key] = env_value

In [31]:
aol_password = env['ARCGIS_GRETA_PASS']
aol_username = env['ARCGIS_GRETA_USER']

In [32]:
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password, profile = "eowilson")

### 20 places geometries

In [43]:
places= gpd.read_file('/Users/tamarahuete/Documents/HALF_EARTH/Top_20_places_per_country/Top_20_places_per_country_20211123.shp')
places.head()

,GID_0,NAME_0,cluster,PRmean,AREA_KM2,geometry,MOL_ID
0,AFG,Afghanistan,1,60.222222,7687.504920,"POLYGON ((6784878.655 4081461.837, 6783664.553...",1
1,AFG,Afghanistan,2,60.222222,4706.088317,"POLYGON ((6833826.327 4162440.126, 6832851.109...",2
2,AFG,Afghanistan,3,52.312500,11768.366965,"POLYGON ((6641029.409 4066345.857, 6640031.675...",3
3,AFG,Afghanistan,4,45.500000,205.123344,"POLYGON ((6544864.953 4033762.857, 6544087.481...",4
4,AFG,Afghanistan,5,45.500000,205.123344,"POLYGON ((6520743.383 4065965.621, 6519965.911...",5


In [42]:
places['MOL_ID']= places.index+1

#### Format Biodiversity data

In [44]:
### BIOdiversity 

dff = places
dff['reptiles']=np.nan
dff['amphibians']=np.nan
dff['mammals']=np.nan
dff['birds']=np.nan


### Paths to data generate inside ArcGIS Pro
paths = {'amphibians':'../../HALF_EARTH/20placescsvs/Sample_amph_nozeroes.csv',
         'birds':'../../HALF_EARTH/20placescsvs/Sample_birds_nozeroes.csv',
         'mammals':'../../HALF_EARTH/20placescsvs/Sample_mammals_nozeroes.csv',
         'reptiles':'../../HALF_EARTH/20placescsvs/Sample_reptiles_nozeroes.csv'}

### Ids of lookup tables for each taxa in ArcGIS online
lookups = {'amphibians':'a641a4cd269345dea93b8bcb1cb66676',
         'birds':'4d8698734b654bb9bb7a61d9af314c76',
         'mammals':'84d3c71caf97479d85f620a4ee217d68',
         'reptiles':'b720e84869624e5482b25338b27a54f2'}

In [29]:
def getHTfromId(item_id):
    item = gis.content.get(item_id)
    flayer = item.tables[0]
    sdf = flayer.query().sdf
    return sdf

In [72]:
df.head(9)

,OID_,MOL_ID,X,Y,reptiles,SliceNumber,Dimensions,range_area_km2,percentage_presence
0,574,124,1.044409e+06,5.381166e+06,2883.0,574.0,SliceNumber,1758510,0.16
1,577,124,1.044409e+06,5.381166e+06,337.0,577.0,SliceNumber,195714,0.17
2,2500,124,1.044409e+06,5.381166e+06,2790.0,2500.0,SliceNumber,5716630,0.05
3,3632,124,1.044409e+06,5.381166e+06,3112.0,3632.0,SliceNumber,7180982,0.04
4,4685,124,1.044409e+06,5.381166e+06,232.0,4685.0,SliceNumber,37097,0.63
5,6311,124,1.044409e+06,5.381166e+06,2258.0,6311.0,SliceNumber,9071053,0.02
6,7275,124,1.044409e+06,5.381166e+06,1383.0,7275.0,SliceNumber,1655955,0.08
7,10241,125,9.450711e+05,5.372708e+06,1493.0,574.0,SliceNumber,1758510,0.08
8,12167,125,9.450711e+05,5.372708e+06,1431.0,2500.0,SliceNumber,5716630,0.03


In [66]:
%%time
### Loop over each set of data and add all results to one dictionary that will be a text field in the Feature Service
for taxa in paths.keys():
    print(taxa) 
    df = pd.read_csv(paths[taxa])
    #col_name = [col for col in df.columns if 'SUM' in col]
    #df.rename(columns={'SliceNumbe':'SliceNumber',col_name[0]:'SUM'}, inplace=True)
    #df = df.merge(gadm[['MOL_ID','AREA_KM2']])
    lookup = getHTfromId(lookups[taxa])
    df = df.merge(lookup[['SliceNumber','range_area_km2']], how='left',on = 'SliceNumber')
    if taxa == 'mammals':
        df['percentage_presence'] = round(df['presence']/df['range_area_km2']*100,2)
    else:
        df['percentage_presence'] = round(df[taxa]/df['range_area_km2']*100,2)
    for i in dff.MOL_ID:
        species = []
        for j in df[(df.MOL_ID== i)].SliceNumber.unique():
            sp_dict = {'SliceNumber':df[(df.MOL_ID== i) & (df.SliceNumber==j)].SliceNumber.values[0].astype(float),
                       'percentage_presence':df[(df.MOL_ID== i) & (df.SliceNumber==j)].percentage_presence.values[0]}
            species.append(sp_dict)
        dff.loc[dff['MOL_ID']==i,taxa] =json.dumps(species)
    print(f'finished MOL_ID ={i}')
pd.set_option('display.max_colwidth',None)
dff.to_file("../../HALF_EARTH/top_places_precalc_biodiv.geojson",driver='GeoJSON')
pd.reset_option('display.max_colwidth')

#### Don't export as shapefile because it truncates the string length

mammals
finished MOL_ID =1925
reptiles
finished MOL_ID =1925
CPU times: user 8min 8s, sys: 9.53 s, total: 8min 17s
Wall time: 8min 51s


In [73]:
dff.head(9)

,GID_0,NAME_0,cluster,PRmean,AREA_KM2,geometry,MOL_ID,reptiles,amphibians,mammals,birds
0,AFG,Afghanistan,1,60.222222,7687.504920,"POLYGON ((6784878.655 4081461.837, 6783664.553...",1,"[{""SliceNumber"": 9.0, ""percentage_presence"": 0...","[{""SliceNumber"": 951.0, ""percentage_presence"":...","[{""SliceNumber"": 167.0, ""percentage_presence"":...","[{""SliceNumber"": 26.0, ""percentage_presence"": ..."
1,AFG,Afghanistan,2,60.222222,4706.088317,"POLYGON ((6833826.327 4162440.126, 6832851.109...",2,"[{""SliceNumber"": 9.0, ""percentage_presence"": 0...","[{""SliceNumber"": 951.0, ""percentage_presence"":...","[{""SliceNumber"": 167.0, ""percentage_presence"":...","[{""SliceNumber"": 92.0, ""percentage_presence"": ..."
2,AFG,Afghanistan,3,52.312500,11768.366965,"POLYGON ((6641029.409 4066345.857, 6640031.675...",3,"[{""SliceNumber"": 1.0, ""percentage_presence"": 0...","[{""SliceNumber"": 32.0, ""percentage_presence"": ...","[{""SliceNumber"": 167.0, ""percentage_presence"":...","[{""SliceNumber"": 26.0, ""percentage_presence"": ..."
3,AFG,Afghanistan,4,45.500000,205.123344,"POLYGON ((6544864.953 4033762.857, 6544087.481...",4,"[{""SliceNumber"": 1.0, ""percentage_presence"": 0...",[],"[{""SliceNumber"": 575.0, ""percentage_presence"":...","[{""SliceNumber"": 26.0, ""percentage_presence"": ..."
4,AFG,Afghanistan,5,45.500000,205.123344,"POLYGON ((6520743.383 4065965.621, 6519965.911...",5,"[{""SliceNumber"": 1.0, ""percentage_presence"": 0...",[],"[{""SliceNumber"": 575.0, ""percentage_presence"":...","[{""SliceNumber"": 26.0, ""percentage_presence"": ..."
5,AFG,Afghanistan,6,45.000000,205.123344,"POLYGON ((6376013.962 3615126.928, 6375236.491...",6,"[{""SliceNumber"": 57.0, ""percentage_presence"": ...","[{""SliceNumber"": 953.0, ""percentage_presence"":...","[{""SliceNumber"": 575.0, ""percentage_presence"":...","[{""SliceNumber"": 123.0, ""percentage_presence"":..."
6,AFG,Afghanistan,7,43.538462,9262.838696,"POLYGON ((6350690.394 3759434.200, 6349476.016...",7,"[{""SliceNumber"": 57.0, ""percentage_presence"": ...","[{""SliceNumber"": 1707.0, ""percentage_presence""...","[{""SliceNumber"": 575.0, ""percentage_presence"":...","[{""SliceNumber"": 26.0, ""percentage_presence"": ..."
7,AFG,Afghanistan,8,42.000000,3949.445557,"POLYGON ((6302387.758 3632855.881, 6301227.474...",8,"[{""SliceNumber"": 57.0, ""percentage_presence"": ...","[{""SliceNumber"": 953.0, ""percentage_presence"":...","[{""SliceNumber"": 575.0, ""percentage_presence"":...","[{""SliceNumber"": 123.0, ""percentage_presence"":..."
8,AFG,Afghanistan,9,42.000000,205.123344,"POLYGON ((6327770.822 3615126.928, 6326993.351...",9,"[{""SliceNumber"": 57.0, ""percentage_presence"": ...","[{""SliceNumber"": 953.0, ""percentage_presence"":...","[{""SliceNumber"": 575.0, ""percentage_presence"":...","[{""SliceNumber"": 423.0, ""percentage_presence"":..."


In [71]:
dff.to_file("../../HALF_EARTH/top_places_precalc_biodiv.geojson",driver='GeoJSON')

In [70]:
dff.loc[dff['MOL_ID']==20,'mammals'].values[0]

'[{"SliceNumber": 575.0, "percentage_presence": 1.62}, {"SliceNumber": 605.0, "percentage_presence": 0.0}, {"SliceNumber": 607.0, "percentage_presence": 0.01}, {"SliceNumber": 609.0, "percentage_presence": 0.0}, {"SliceNumber": 615.0, "percentage_presence": 0.04}, {"SliceNumber": 635.0, "percentage_presence": 0.0}, {"SliceNumber": 1176.0, "percentage_presence": 0.09}, {"SliceNumber": 1613.0, "percentage_presence": 0.05}, {"SliceNumber": 1784.0, "percentage_presence": 0.0}, {"SliceNumber": 1787.0, "percentage_presence": 0.0}, {"SliceNumber": 1955.0, "percentage_presence": 0.01}, {"SliceNumber": 2106.0, "percentage_presence": 0.01}, {"SliceNumber": 2240.0, "percentage_presence": 0.01}, {"SliceNumber": 2335.0, "percentage_presence": 0.01}, {"SliceNumber": 2341.0, "percentage_presence": 0.0}, {"SliceNumber": 2811.0, "percentage_presence": 0.17}, {"SliceNumber": 2889.0, "percentage_presence": 0.0}, {"SliceNumber": 2927.0, "percentage_presence": 0.04}, {"SliceNumber": 3041.0, "percentage_pre

### Format contextual data 

In [77]:
### Read files (Files coming from ArcGIS project Places_HE_Future and expoprted as csv)
### Top 20 places original geometries from Scott: https://eowilson.maps.arcgis.com/home/item.html?id=4848c6b08fac4fa5bff40e9331b6d291
elu= pd.read_csv('/Users/tamarahuete/Documents/HALF_EARTH/20placescsvs/Places_elu.csv')
le= pd.read_csv('/Users/tamarahuete/Documents/HALF_EARTH/20placescsvs/Places_encroachment.csv')
pop= pd.read_csv('/Users/tamarahuete/Documents/HALF_EARTH/20placescsvs/Places_pop.csv')
wl= pd.read_csv('/Users/tamarahuete/Documents/HALF_EARTH/20placescsvs/Places_wdpa_list.csv')
wp= pd.read_csv('/Users/tamarahuete/Documents/HALF_EARTH/20placescsvs/Places_wdpa_percentage.csv')

cont_lookups={'elu':'83802a7fa3d34c1fa40844fc14683966',
              'landencroach':'b0ad58e2b2404eec811b711b604b9d55'}

In [75]:
### Column names needed: 
'percent_irrigated', 'percent_rainfed',
       'percent_rangeland', 'percent_urban', 'population_sum',
       'majority_land_cover_climate_regime', 'land_cover_majority',
       'climate_regime_majority',

Index(['Unnamed: 0', 'GID_0', 'NAME_0', 'GID_1', 'NAME_1', 'MOL_ID',
       'AREA_KM2', 'reptiles', 'amphibians', 'mammals', 'birds',
       'percentage_protected', 'percent_irrigated', 'percent_rainfed',
       'percent_rangeland', 'percent_urban', 'population_sum',
       'majority_land_cover_climate_regime', 'land_cover_majority',
       'climate_regime_majority', 'country_size'],
      dtype='object')

In [74]:
### Add contextual data
elu.head()

,OID_,OBJECTID_1,COUNT,AREA,MAJORITY
0,1,1,148821.0,0.750596,184
1,2,2,92178.0,0.464910,176
2,3,3,229561.0,1.157817,142
3,4,4,3949.0,0.019917,142
4,5,5,3957.0,0.019958,142


In [84]:
dff = dff.merge(elu[['OBJECTID_1','MAJORITY']], how='left', left_on = 'MOL_ID', right_on = 'OBJECTID_1').drop(columns=['OBJECTID_1']).rename(columns={'MAJORITY':'majority_land_cover_climate_regime'})

In [86]:
dff.merge(elu_lookup[['elu_code','lc_type','cr_type']], how='left', left_on = 'majority_land_cover_climate_regime', right_on = 'elu_code')\
    .drop(columns=['elu_code'])\
    .rename(columns={'lc_type':'land_cover_majority','cr_type':'climate_regime_majority'})

,GID_0,NAME_0,cluster,PRmean,AREA_KM2,geometry,MOL_ID,reptiles,amphibians,mammals,birds,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority
0,AFG,Afghanistan,1,60.222222,7687.504920,"POLYGON ((6784878.655 4081461.837, 6783664.553...",1,"[{""SliceNumber"": 9.0, ""percentage_presence"": 0...","[{""SliceNumber"": 951.0, ""percentage_presence"":...","[{""SliceNumber"": 167.0, ""percentage_presence"":...","[{""SliceNumber"": 26.0, ""percentage_presence"": ...",184.0,Cropland,Warm Temperate Dry
1,AFG,Afghanistan,2,60.222222,4706.088317,"POLYGON ((6833826.327 4162440.126, 6832851.109...",2,"[{""SliceNumber"": 9.0, ""percentage_presence"": 0...","[{""SliceNumber"": 951.0, ""percentage_presence"":...","[{""SliceNumber"": 167.0, ""percentage_presence"":...","[{""SliceNumber"": 92.0, ""percentage_presence"": ...",176.0,Forest,Warm Temperate Moist
2,AFG,Afghanistan,3,52.312500,11768.366965,"POLYGON ((6641029.409 4066345.857, 6640031.675...",3,"[{""SliceNumber"": 1.0, ""percentage_presence"": 0...","[{""SliceNumber"": 32.0, ""percentage_presence"": ...","[{""SliceNumber"": 167.0, ""percentage_presence"":...","[{""SliceNumber"": 26.0, ""percentage_presence"": ...",142.0,Grassland,Cool Temperate Dry
3,AFG,Afghanistan,4,45.500000,205.123344,"POLYGON ((6544864.953 4033762.857, 6544087.481...",4,"[{""SliceNumber"": 1.0, ""percentage_presence"": 0...",[],"[{""SliceNumber"": 575.0, ""percentage_presence"":...","[{""SliceNumber"": 26.0, ""percentage_presence"": ...",142.0,Grassland,Cool Temperate Dry
4,AFG,Afghanistan,5,45.500000,205.123344,"POLYGON ((6520743.383 4065965.621, 6519965.911...",5,"[{""SliceNumber"": 1.0, ""percentage_presence"": 0...",[],"[{""SliceNumber"": 575.0, ""percentage_presence"":...","[{""SliceNumber"": 26.0, ""percentage_presence"": ...",142.0,Grassland,Cool Temperate Dry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920,ZWE,Zimbabwe,1921,27.000000,205.123344,"POLYGON ((2492441.182 -2277978.839, 2491663.71...",1921,"[{""SliceNumber"": 168.0, ""percentage_presence"":...","[{""SliceNumber"": 212.0, ""percentage_presence"":...","[{""SliceNumber"": 28.0, ""percentage_presence"": ...","[{""SliceNumber"": 26.0, ""percentage_presence"": ...",285.0,Shrubland,Sub Tropical Dry
1921,ZWE,Zimbabwe,1922,26.000000,205.123344,"POLYGON ((2733656.883 -2664412.005, 2732879.41...",1922,"[{""SliceNumber"": 168.0, ""percentage_presence"":...","[{""SliceNumber"": 212.0, ""percentage_presence"":...","[{""SliceNumber"": 28.0, ""percentage_presence"": ...","[{""SliceNumber"": 26.0, ""percentage_presence"": ...",NaN,NaN,NaN
1922,ZWE,Zimbabwe,1923,23.500000,862.495051,"POLYGON ((2436278.576 -2223713.158, 2438146.83...",1923,"[{""SliceNumber"": 168.0, ""percentage_presence"":...","[{""SliceNumber"": 212.0, ""percentage_presence"":...","[{""SliceNumber"": 28.0, ""percentage_presence"": ...","[{""SliceNumber"": 26.0, ""percentage_presence"": ...",NaN,NaN,NaN
1923,ZWE,Zimbabwe,1924,1.000000,205.123344,"POLYGON ((2516562.752 -2374587.131, 2515785.28...",1924,"[{""SliceNumber"": 168.0, ""percentage_presence"":...","[{""SliceNumber"": 212.0, ""percentage_presence"":...","[{""SliceNumber"": 28.0, ""percentage_presence"": ...","[{""SliceNumber"": 26.0, ""percentage_presence"": ...",NaN,NaN,NaN


In [89]:
dff[dff['majority_land_cover_climate_regime'].isna()]

,GID_0,NAME_0,cluster,PRmean,AREA_KM2,geometry,MOL_ID,reptiles,amphibians,mammals,birds,majority_land_cover_climate_regime
26,DZA,Algeria,27,40.900000,205.123344,"POLYGON ((80284.176 4355790.494, 79506.705 435...",27,[],[],[],"[{""SliceNumber"": 462.0, ""percentage_presence"":...",NaN
61,ATA,Antarctica,62,11.896104,179550.043336,"POLYGON ((-6238396.930 -6820380.816, -6237086....",62,[],[],[],[],NaN
62,ATA,Antarctica,63,10.472727,41941.512891,"POLYGON ((9840712.596 -6730377.625, 9840672.35...",63,[],[],[],"[{""SliceNumber"": 2915.0, ""percentage_presence""...",NaN
63,ATA,Antarctica,64,10.380952,15523.349652,"POLYGON ((14916179.704 -6884608.322, 14914845....",64,[],[],[],[],NaN
64,ATA,Antarctica,65,7.708333,36613.756454,"POLYGON ((13121381.878 -6730377.625, 13121345....",65,[],[],[],"[{""SliceNumber"": 2915.0, ""percentage_presence""...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1869,ESH,Western Sahara,1870,18.111111,205.123344,"POLYGON ((-1608225.729 2777855.070, -1609003.2...",1870,[],[],"[{""SliceNumber"": 3171.0, ""percentage_presence""...","[{""SliceNumber"": 1194.0, ""percentage_presence""...",NaN
1921,ZWE,Zimbabwe,1922,26.000000,205.123344,"POLYGON ((2733656.883 -2664412.005, 2732879.41...",1922,"[{""SliceNumber"": 168.0, ""percentage_presence"":...","[{""SliceNumber"": 212.0, ""percentage_presence"":...","[{""SliceNumber"": 28.0, ""percentage_presence"": ...","[{""SliceNumber"": 26.0, ""percentage_presence"": ...",NaN
1922,ZWE,Zimbabwe,1923,23.500000,862.495051,"POLYGON ((2436278.576 -2223713.158, 2438146.83...",1923,"[{""SliceNumber"": 168.0, ""percentage_presence"":...","[{""SliceNumber"": 212.0, ""percentage_presence"":...","[{""SliceNumber"": 28.0, ""percentage_presence"": ...","[{""SliceNumber"": 26.0, ""percentage_presence"": ...",NaN
1923,ZWE,Zimbabwe,1924,1.000000,205.123344,"POLYGON ((2516562.752 -2374587.131, 2515785.28...",1924,"[{""SliceNumber"": 168.0, ""percentage_presence"":...","[{""SliceNumber"": 212.0, ""percentage_presence"":...","[{""SliceNumber"": 28.0, ""percentage_presence"": ...","[{""SliceNumber"": 26.0, ""percentage_presence"": ...",NaN


In [78]:
elu_lookup = getHTfromId(cont_lookups['elu'])

In [79]:
elu_lookup.head()

,elu_code,elu,lc_type,lf_type,cr_type,ObjectId
0,301,Sub Tropical Moist Forest on Plains,Forest,Plains,Sub Tropical Moist,1
1,201,Warm Temperate Dry Sparsley or Non vegetated o...,Sparsley or Non vegetated,Plains,Warm Temperate Dry,2
2,151,Cool Temperate Dry Sparsley or Non vegetated o...,Sparsley or Non vegetated,Plains,Cool Temperate Dry,3
3,302,Sub Tropical Moist Cropland on Tablelands,Cropland,Tablelands,Sub Tropical Moist,4
4,152,Cool Temperate Dry Sparsley or Non vegetated o...,Sparsley or Non vegetated,Tablelands,Cool Temperate Dry,5
5,351,Sub Tropical Desert Settlement on Mountains,Settlement,Mountains,Sub Tropical Desert,6
6,202,Warm Temperate Dry Forest on Tablelands,Forest,Tablelands,Warm Temperate Dry,7
7,101,Cool Temperate Moist Forest on Tablelands,Forest,Tablelands,Cool Temperate Moist,8
8,303,Sub Tropical Moist Forest on Tablelands,Forest,Tablelands,Sub Tropical Moist,9
